# Testing the Trained Gradient boosting model on the Test Data Set

## Data Preprocessing and Prediction

Now, we are going to take the test dataset and preprocess the data exactly the same way as the trained dataset. We'll apply the same scaler that was used during the training phase. After preprocessing, we will use the trained Gradient Boosting model to make predictions and export the predictions in excel file .

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

In [2]:
# options to display all the columns in the dataframe
pd.set_option('display.max_columns', None)

In [3]:
# Loading the testing Dataset
df_test = pd.read_excel("House_Rent_Test.xlsx")

In [4]:
# Displying the top 5 rows of the testing dataset
df_test.head()

,id,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies
0,ff8081815df539bc015df947ce976cca,BHK2,Basavanagudi,22-08-2017 09:00,12.941603,77.568156,FAMILY,0,1,0,0,SEMI_FURNISHED,BOTH,1300,1,2,N,2,3,3,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",CORP_BORE,AP,1
1,ff80818157288e9301572c05651853a6,BHK2,Rajaji Nagar,17-09-2017 16:33,12.998803,77.561887,ANYONE,0,0,0,1,SEMI_FURNISHED,TWO_WHEELER,600,7,1,S,2,0,3,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",CORPORATION,IF,0
2,ff8081815f1afc58015f1b831fde166e,BHK1,Jeevan Bima Nagar,14-04-2018 17:27,12.966467,77.661063,ANYONE,0,0,0,1,SEMI_FURNISHED,TWO_WHEELER,600,10,1,S,1,0,1,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",CORPORATION,IF,0
3,ff8081816035b31901603f7922c046e0,BHK1,Jayanagar 1st Block,2017-11-12 13:15:00,12.941533,77.592606,ANYONE,0,0,0,0,NOT_FURNISHED,TWO_WHEELER,500,8,1,E,0,2,2,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",CORP_BORE,IF,0
4,ff8081815c3582f5015c38fc31aa73a4,BHK3,Whitefield Hope Farm Junction,2017-11-07 16:44:00,12.971083,77.751625,ANYONE,1,1,1,1,SEMI_FURNISHED,BOTH,1400,0,3,E,3,3,4,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",BOREWELL,AP,1


In [5]:
# To see what is the shape of the testing dataset

print("No of Rows :", df_test.shape[0])
print("No of Columns:", df_test.shape[1])

No of Rows : 4500
No of Columns: 24


# Calculating the missing percentage of values for the testing dataset

In [6]:
missing_values = df_test.isnull().sum()
missing_percentages = (missing_values / len(df_test)) * 100

missing_info = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentages
})

# Display the missing values and percentages
print(missing_info)

                 Missing Values  Percentage
id                            0         0.0
type                          0         0.0
locality                      0         0.0
activation_date               0         0.0
latitude                      0         0.0
longitude                     0         0.0
lease_type                    0         0.0
gym                           0         0.0
lift                          0         0.0
swimming_pool                 0         0.0
negotiable                    0         0.0
furnishing                    0         0.0
parking                       0         0.0
property_size                 0         0.0
property_age                  0         0.0
bathroom                      0         0.0
facing                        0         0.0
cup_board                     0         0.0
floor                         0         0.0
total_floor                   0         0.0
amenities                     0         0.0
water_supply                  0 

# Handling Duplicates in the traing and testing data

In [7]:
duplicate_rows = df_test[df_test.duplicated()]

In [8]:
duplicate_rows

,id,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies


# Droping the ID column since it is a unique identifier for each property

In [9]:
# Drop the 'id' column
df_test = df_test.drop('id', axis=1)

In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   type             4500 non-null   object 
 1   locality         4500 non-null   object 
 2   activation_date  4500 non-null   object 
 3   latitude         4500 non-null   float64
 4   longitude        4500 non-null   float64
 5   lease_type       4500 non-null   object 
 6   gym              4500 non-null   int64  
 7   lift             4500 non-null   int64  
 8   swimming_pool    4500 non-null   int64  
 9   negotiable       4500 non-null   int64  
 10  furnishing       4500 non-null   object 
 11  parking          4500 non-null   object 
 12  property_size    4500 non-null   int64  
 13  property_age     4500 non-null   int64  
 14  bathroom         4500 non-null   int64  
 15  facing           4500 non-null   object 
 16  cup_board        4500 non-null   int64  
 17  floor         

# Data Preprocessing

# Checking type column 

In [11]:
df_test['type'].unique()

array(['BHK2', 'BHK1', 'BHK3', 'RK1', 'BHK4', 'BHK4PLUS'], dtype=object)

In [12]:
df_test['activation_date'].unique()

array(['22-08-2017 09:00', '17-09-2017 16:33', '14-04-2018 17:27', ...,
       '30-05-2018 16:10', datetime.datetime(2018, 11, 7, 16, 17),
       '27-03-2018 16:25'], dtype=object)

In [13]:
df_test['activation_date'] = pd.to_datetime(df_test['activation_date'], errors='coerce')

# Extract only the date portion
df_test['activation_date'] = df_test['activation_date'].dt.date


C:\Users\DIVAHAR\AppData\Local\Temp\ipykernel_3168\4217020189.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_test['activation_date'] = pd.to_datetime(df_test['activation_date'], errors='coerce')


In [14]:
df_test['activation_date'].unique()

array([datetime.date(2017, 8, 22), datetime.date(2017, 9, 17),
       datetime.date(2018, 4, 14), datetime.date(2017, 11, 12),
       datetime.date(2017, 11, 7), datetime.date(2017, 12, 31),
       datetime.date(2017, 6, 12), datetime.date(2017, 4, 9),
       datetime.date(2018, 4, 3), datetime.date(2018, 5, 20),
       datetime.date(2018, 5, 18), datetime.date(2017, 9, 14),
       datetime.date(2018, 7, 29), datetime.date(2018, 9, 3),
       datetime.date(2018, 10, 7), datetime.date(2017, 8, 7),
       datetime.date(2018, 6, 20), datetime.date(2017, 9, 25),
       datetime.date(2018, 12, 1), datetime.date(2017, 9, 20),
       datetime.date(2017, 10, 30), datetime.date(2017, 12, 13),
       datetime.date(2017, 10, 12), datetime.date(2018, 2, 7),
       datetime.date(2018, 12, 6), datetime.date(2017, 8, 28),
       datetime.date(2018, 1, 22), datetime.date(2018, 7, 25),
       datetime.date(2018, 7, 18), datetime.date(2017, 10, 16),
       datetime.date(2018, 5, 15), datetime.date(2018,

In [15]:
df_test['latitude'].unique()

array([12.94160313, 12.99880262, 12.96646692, ..., 12.94297279,
       12.94931477, 12.91529639])

In [16]:
df_test['longitude'].unique()

array([77.56815588, 77.56188704, 77.66106338, ..., 77.59773779,
       77.713807  , 77.67669355])

In [17]:
df_test['lease_type'].unique()

array(['FAMILY', 'ANYONE', 'BACHELOR', 'COMPANY'], dtype=object)

In [18]:
df_test['gym'].unique()

array([0, 1], dtype=int64)

In [19]:
df_test['lift'].unique()

array([1, 0], dtype=int64)

In [20]:
df_test['swimming_pool'].unique()

array([0, 1], dtype=int64)

In [21]:
df_test['negotiable'].unique()

array([0, 1], dtype=int64)

In [22]:
df_test['furnishing'].unique()

array(['SEMI_FURNISHED', 'NOT_FURNISHED', 'FULLY_FURNISHED'], dtype=object)

In [23]:
df_test['facing'].unique()

array(['N', 'S', 'E', 'W', 'NE', 'NW', 'SE', 'SW'], dtype=object)

In [24]:
df_test['water_supply'].unique()

array(['CORP_BORE', 'CORPORATION', 'BOREWELL'], dtype=object)

In [25]:
df_test['building_type'].unique()

array(['AP', 'IF', 'IH', 'GC'], dtype=object)

In [26]:
df_test['parking'].unique()

array(['BOTH', 'TWO_WHEELER', 'FOUR_WHEELER', 'NONE'], dtype=object)

#  Extracting and Analyzing Amenities Data

In [27]:
from pandas import json_normalize

# Parse the 'amenities' column as JSON, skipping invalid JSON or NaN values
df_test['amenities'] = df_test['amenities'].apply(lambda x: json.loads(str(x).replace("'", "\"")) if pd.notnull(x) else {})

# Normalize the 'amenities' column
amenities_df = json_normalize(df_test['amenities'])

In [28]:
amenities_df.head()

,LIFT,GYM,INTERNET,AC,CLUB,INTERCOM,POOL,CPA,FS,SERVANT,SECURITY,SC,GP,PARK,RWH,STP,HK,PB,VP
0,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,False
1,False,False,True,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,True,False,True,True,False,False,True,False
3,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,True,True,True,False,True,True,True,True,True,False,True,False,False,True,True,True,True,True,True


In [29]:
# Display unique values, counts, and count of NaN for each column in amenities_df
for column in amenities_df.columns:
    unique_counts = amenities_df[column].value_counts()
    nan_count = amenities_df[column].isna().sum()
    print(f"Column: {column}\nUnique Values:\n{unique_counts}\nNaN Count: {nan_count}\n")

Column: LIFT
Unique Values:
LIFT
False    2715
True     1785
Name: count, dtype: int64
NaN Count: 0

Column: GYM
Unique Values:
GYM
False    3392
True     1088
Name: count, dtype: int64
NaN Count: 20

Column: INTERNET
Unique Values:
INTERNET
True     3018
False    1482
Name: count, dtype: int64
NaN Count: 0

Column: AC
Unique Values:
AC
False    4297
True      203
Name: count, dtype: int64
NaN Count: 0

Column: CLUB
Unique Values:
CLUB
False    3417
True      867
Name: count, dtype: int64
NaN Count: 216

Column: INTERCOM
Unique Values:
INTERCOM
False    3584
True      916
Name: count, dtype: int64
NaN Count: 0

Column: POOL
Unique Values:
POOL
False    3656
True      844
Name: count, dtype: int64
NaN Count: 0

Column: CPA
Unique Values:
CPA
False    2875
True     1409
Name: count, dtype: int64
NaN Count: 216

Column: FS
Unique Values:
FS
False    3423
True     1077
Name: count, dtype: int64
NaN Count: 0

Column: SERVANT
Unique Values:
SERVANT
False    4073
True      211
Name: count, dt

In [30]:
# Replace True with 1 and False with 0 for all columns
amenities_df = amenities_df.applymap(lambda x: 1 if x == True else 0)

In [31]:
amenities_df.head()

,LIFT,GYM,INTERNET,AC,CLUB,INTERCOM,POOL,CPA,FS,SERVANT,SECURITY,SC,GP,PARK,RWH,STP,HK,PB,VP
0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1,1,1,0,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1


In [32]:
amenities_df.shape

(4500, 19)

In [33]:
amenities_df.isnull().sum()

LIFT        0
GYM         0
INTERNET    0
AC          0
CLUB        0
INTERCOM    0
POOL        0
CPA         0
FS          0
SERVANT     0
SECURITY    0
SC          0
GP          0
PARK        0
RWH         0
STP         0
HK          0
PB          0
VP          0
dtype: int64

In [34]:
# Assuming amenities_df is your DataFrame
columns_to_drop = ['SC', 'GP', 'STP', 'PB', 'VP', 'LIFT', 'GYM', 'POOL','CPA','FS','RWH']

# Drop the specified columns
amenities_df = amenities_df.drop(columns=columns_to_drop)

In [35]:
amenities_df.rename(columns={'HK': 'HOUSE_KEEPING'}, inplace=True)

In [36]:
amenities_df.head()

,INTERNET,AC,CLUB,INTERCOM,SERVANT,SECURITY,PARK,HOUSE_KEEPING
0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0
4,1,0,1,1,0,1,1,1


In [37]:
amenities_df.shape

(4500, 8)

In [38]:
df_test.shape

(4500, 23)

In [39]:
df_test = pd.merge(df_test, amenities_df, left_index=True, right_index=True)


In [40]:
df_test.isnull().sum()

type               0
locality           0
activation_date    0
latitude           0
longitude          0
lease_type         0
gym                0
lift               0
swimming_pool      0
negotiable         0
furnishing         0
parking            0
property_size      0
property_age       0
bathroom           0
facing             0
cup_board          0
floor              0
total_floor        0
amenities          0
water_supply       0
building_type      0
balconies          0
INTERNET           0
AC                 0
CLUB               0
INTERCOM           0
SERVANT            0
SECURITY           0
PARK               0
HOUSE_KEEPING      0
dtype: int64

In [41]:
# Drop the 'amenities' column
df_test = df_test.drop('amenities', axis=1)

In [42]:
df_test["activation_date"]

0       2017-08-22
1       2017-09-17
2       2018-04-14
3       2017-11-12
4       2017-11-07
           ...    
4495    2018-10-06
4496    2018-01-21
4497    2018-05-30
4498    2018-11-07
4499    2018-03-27
Name: activation_date, Length: 4500, dtype: object

In [43]:
df_test["activation_date"] = pd.to_datetime(df_test["activation_date"])

In [44]:
print(df_test["activation_date"].dtype)

datetime64[ns]


In [45]:
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   type             4500 non-null   object        
 1   locality         4500 non-null   object        
 2   activation_date  4500 non-null   datetime64[ns]
 3   latitude         4500 non-null   float64       
 4   longitude        4500 non-null   float64       
 5   lease_type       4500 non-null   object        
 6   gym              4500 non-null   int64         
 7   lift             4500 non-null   int64         
 8   swimming_pool    4500 non-null   int64         
 9   negotiable       4500 non-null   int64         
 10  furnishing       4500 non-null   object        
 11  parking          4500 non-null   object        
 12  property_size    4500 non-null   int64         
 13  property_age     4500 non-null   int64         
 14  bathroom         4500 non-null   int64  

In [46]:
# Drop the 'amenities' column
df_test = df_test.drop('locality', axis=1)

In [47]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   type             4500 non-null   object        
 1   activation_date  4500 non-null   datetime64[ns]
 2   latitude         4500 non-null   float64       
 3   longitude        4500 non-null   float64       
 4   lease_type       4500 non-null   object        
 5   gym              4500 non-null   int64         
 6   lift             4500 non-null   int64         
 7   swimming_pool    4500 non-null   int64         
 8   negotiable       4500 non-null   int64         
 9   furnishing       4500 non-null   object        
 10  parking          4500 non-null   object        
 11  property_size    4500 non-null   int64         
 12  property_age     4500 non-null   int64         
 13  bathroom         4500 non-null   int64         
 14  facing           4500 non-null   object 

# Label Encoding for the categorical Variables

In [48]:
# Label encoding for 'type'
type_encoding = {
    'RK1': 0,
    'BHK1': 1,
    'BHK2': 2,
    'BHK3': 3,
    'BHK4': 4,
    'BHK4PLUS': 5
}

# Label encoding for 'lease_type'
lease_type_encoding = {
    'ANYONE': 1,
    'FAMILY': 2,
    'BACHELOR': 0,
    'COMPANY': 3
}

# Label encoding for 'furnishing'
furnishing_encoding = {
    'NOT_FURNISHED': 0,
    'SEMI_FURNISHED': 1,
    'FULLY_FURNISHED': 2
}

# Label encoding for 'parking'
parking_encoding = {
    'TWO_WHEELER': 0,
    'NONE': 1,
    'FOUR_WHEELER': 2,
    'BOTH': 3
}

# Label encoding for 'facing'
facing_encoding = {
    'N': 0,
    'E': 1,
    'W': 2,
    'NE': 3,
    'NW': 4,
    'S': 5,
    'SE': 6,
    'SW': 7
}

# Label encoding for 'water_supply'
water_supply_encoding = {
    'CORPORATION': 0,
    'CORP_BORE': 1,
    'BOREWELL': 2
}

# Label encoding for 'building_type'
building_type_encoding = {
    'IF': 0,
    'IH': 1,
    'AP': 2,
    'GC': 3
}

# Apply label encoding to your DataFrame
df_test['type'] = df_test['type'].map(type_encoding).astype('int64')
df_test['lease_type'] = df_test['lease_type'].map(lease_type_encoding).astype('int64')
df_test['furnishing'] = df_test['furnishing'].map(furnishing_encoding).astype('int64')
df_test['parking'] = df_test['parking'].map(parking_encoding).astype('int64')
df_test['facing'] = df_test['facing'].map(facing_encoding).astype('int64')
df_test['water_supply'] = df_test['water_supply'].map(water_supply_encoding).astype('int64')
df_test['building_type'] = df_test['building_type'].map(building_type_encoding).astype('int64')


In [49]:
df_test['activation_date'] = pd.to_datetime(df_test['activation_date'])
df_test['day'] = df_test['activation_date'].dt.day
df_test['month'] = df_test['activation_date'].dt.month
df_test['year'] = df_test['activation_date'].dt.year


In [50]:
df_test.head()

,type,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,INTERNET,AC,CLUB,INTERCOM,SERVANT,SECURITY,PARK,HOUSE_KEEPING,day,month,year
0,2,2017-08-22,12.941603,77.568156,2,0,1,0,0,1,3,1300,1,2,0,2,3,3,1,2,1,0,0,0,0,0,1,0,0,22,8,2017
1,2,2017-09-17,12.998803,77.561887,1,0,0,0,1,1,0,600,7,1,5,2,0,3,0,0,0,1,0,0,0,0,0,1,0,17,9,2017
2,1,2018-04-14,12.966467,77.661063,1,0,0,0,1,1,0,600,10,1,5,1,0,1,0,0,0,1,0,0,0,0,0,1,0,14,4,2018
3,1,2017-11-12,12.941533,77.592606,1,0,0,0,0,0,0,500,8,1,1,0,2,2,1,0,0,1,0,0,0,0,0,0,0,12,11,2017
4,3,2017-11-07,12.971083,77.751625,1,1,1,1,1,1,3,1400,0,3,1,3,3,4,2,2,1,1,0,1,1,0,1,1,1,7,11,2017


In [51]:
# Drop the original "activation_date" column
df_test = df_test.drop('activation_date', axis=1)

In [52]:
df_test.head()

,type,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,INTERNET,AC,CLUB,INTERCOM,SERVANT,SECURITY,PARK,HOUSE_KEEPING,day,month,year
0,2,12.941603,77.568156,2,0,1,0,0,1,3,1300,1,2,0,2,3,3,1,2,1,0,0,0,0,0,1,0,0,22,8,2017
1,2,12.998803,77.561887,1,0,0,0,1,1,0,600,7,1,5,2,0,3,0,0,0,1,0,0,0,0,0,1,0,17,9,2017
2,1,12.966467,77.661063,1,0,0,0,1,1,0,600,10,1,5,1,0,1,0,0,0,1,0,0,0,0,0,1,0,14,4,2018
3,1,12.941533,77.592606,1,0,0,0,0,0,0,500,8,1,1,0,2,2,1,0,0,1,0,0,0,0,0,0,0,12,11,2017
4,3,12.971083,77.751625,1,1,1,1,1,1,3,1400,0,3,1,3,3,4,2,2,1,1,0,1,1,0,1,1,1,7,11,2017


In [53]:

# Create a dummy 'rent' column with any values (it won't be used for prediction)
df_test['rent'] = 0  # You can set it to any constant value or use NaN

In [54]:
df_test.head()

,type,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,INTERNET,AC,CLUB,INTERCOM,SERVANT,SECURITY,PARK,HOUSE_KEEPING,day,month,year,rent
0,2,12.941603,77.568156,2,0,1,0,0,1,3,1300,1,2,0,2,3,3,1,2,1,0,0,0,0,0,1,0,0,22,8,2017,0
1,2,12.998803,77.561887,1,0,0,0,1,1,0,600,7,1,5,2,0,3,0,0,0,1,0,0,0,0,0,1,0,17,9,2017,0
2,1,12.966467,77.661063,1,0,0,0,1,1,0,600,10,1,5,1,0,1,0,0,0,1,0,0,0,0,0,1,0,14,4,2018,0
3,1,12.941533,77.592606,1,0,0,0,0,0,0,500,8,1,1,0,2,2,1,0,0,1,0,0,0,0,0,0,0,12,11,2017,0
4,3,12.971083,77.751625,1,1,1,1,1,1,3,1400,0,3,1,3,3,4,2,2,1,1,0,1,1,0,1,1,1,7,11,2017,0


# Importing the scaler pickle file and scaling of the features 

In [55]:


# Load the scaler
with open('minmax_scaler.pkl', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

# List of numerical features used for scaling during training
numerical_features_training = ['latitude', 'longitude', 'property_size', 'property_age', 'bathroom', 'cup_board', 'floor', 'total_floor', 'balconies', 'rent', 'day', 'month', 'year']

# Extract numerical features from the testing data
numerical_features_testing = [col for col in df_test.columns if col in numerical_features_training]

# Ensure the order of features in the testing data matches the order during training
numerical_features_testing = [feature for feature in numerical_features_training if feature in numerical_features_testing]

# Apply the scaler only to the numerical features in the testing data
df_test[numerical_features_testing] = scaler.transform(df_test[numerical_features_testing])

# Remove the dummy 'rent' column
df_test = df_test.drop('rent', axis=1)


In [56]:
df_test.head()

,type,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,INTERNET,AC,CLUB,INTERCOM,SERVANT,SECURITY,PARK,HOUSE_KEEPING,day,month,year
0,2,0.416016,0.024612,2,0,1,0,0,1,3,0.202020,0.02,0.166667,0,0.050,0.12,0.115385,1,2,0.166667,0,0,0,0,0,1,0,0,0.700000,0.636364,0.0
1,2,0.988041,0.022346,1,0,0,0,1,1,0,0.084175,0.14,0.000000,5,0.050,0.00,0.115385,0,0,0.000000,1,0,0,0,0,0,1,0,0.533333,0.727273,0.0
2,1,0.664667,0.058198,1,0,0,0,1,1,0,0.084175,0.20,0.000000,5,0.025,0.00,0.038462,0,0,0.000000,1,0,0,0,0,0,1,0,0.433333,0.272727,1.0
3,1,0.415319,0.033451,1,0,0,0,0,0,0,0.067340,0.16,0.000000,1,0.000,0.08,0.076923,1,0,0.000000,1,0,0,0,0,0,0,0,0.366667,0.909091,0.0
4,3,0.710827,0.090936,1,1,1,1,1,1,3,0.218855,0.00,0.333333,1,0.075,0.12,0.153846,2,2,0.166667,1,0,1,1,0,1,1,1,0.200000,0.909091,0.0


In [57]:
df_test.describe()

,type,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,INTERNET,AC,CLUB,INTERCOM,SERVANT,SECURITY,PARK,HOUSE_KEEPING,day,month,year
count,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000
mean,2.027778,0.465762,0.049934,1.479556,0.223111,0.366889,0.173111,0.719333,0.997556,1.792222,0.163734,0.116542,0.145778,1.210667,0.056106,0.076729,0.148111,0.816667,0.994667,0.194407,0.670667,0.045111,0.192667,0.203556,0.046889,0.468667,0.516889,0.201111,0.485948,0.493556,0.620667
std,0.751605,0.296958,0.042230,0.552454,0.416379,0.482009,0.378385,0.449375,0.368513,1.390017,0.099334,0.316018,0.158430,1.354673,0.040258,0.088046,0.125912,0.663692,0.946428,0.179152,0.470023,0.207571,0.394437,0.402687,0.211424,0.499073,0.499770,0.400875,0.298613,0.293764,0.485275
min,0.000000,0.000032,0.000009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.016667,-0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.194037,0.026004,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.101010,0.020000,0.000000,0.000000,0.025000,0.040000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.272727,0.000000
50%,2.000000,0.444227,0.048033,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,3.000000,0.162290,0.100000,0.166667,1.000000,0.050000,0.040000,0.115385,1.000000,1.000000,0.166667,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.466667,0.545455,1.000000
75%,2.000000,0.717171,0.071272,2.000000,0.000000,1.000000,0.000000,1.000000,1.000000,3.000000,0.202020,0.180000,0.166667,1.000000,0.075000,0.120000,0.153846,1.000000,2.000000,0.333333,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.766667,0.727273,1.000000
max,5.000000,0.999305,0.996808,3.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,3.350168,19.980000,5.500000,7.000000,0.750000,0.840000,1.923077,2.000000,3.000000,3.666667,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Imoporting the trained model and perfoming predictions on the test data

In [58]:
import pickle

# Load the Gradient Boosting model
with open('gradient_boosting_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Make predictions
scaled_predictions = loaded_model.predict(df_test)

scaled_predictions


array([0.40453112, 0.10892555, 0.08462615, ..., 0.06256734, 0.38319192,
       0.55108697])

In [59]:
# Exporting the predictions as a excel file 

In [60]:
# Create a DataFrame with the scaled predictions
df_scaled_predictions = pd.DataFrame({'scaled_rent': scaled_predictions})

# Export the DataFrame to an Excel file
df_scaled_predictions.to_excel('scaled_predictions.xlsx', index=False)